#Objetivo: separar clientes no sistema medallion

##Aquisição dos dados

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('RFV_Telecom.xlsx')

In [3]:
df

,CustomerID,fullName,Genre,PurchaseDate,Product,Quantity,Value
0,11,Linda Rogers,Feminino,2022-11-10,Renovação Plano Mensal,3,150.00
1,80,Antonio Rodriguez,Masculino,2023-03-16,Adição Pacote de Dados (GB),1,10.00
2,166,Randall Wright,Feminino,2022-11-25,SMS,1,0.05
3,131,Patrick Martin,Masculino,2023-08-21,Adição Pacote de Dados (GB),3,30.00
4,175,John Herrera,Masculino,2024-02-23,Adição Pacote de Dados (GB),2,20.00
...,...,...,...,...,...,...,...
995,113,Eric Morales,Feminino,2023-11-05,Adição Pacote de Dados (GB),5,50.00
996,134,Thomas Sexton,Masculino,2023-08-22,SMS,5,0.25
997,40,Melissa Farrell,Masculino,2024-01-31,Renovação Plano Mensal,5,250.00
998,10,Kimberly Nicholson,Feminino,2023-06-28,Adição Pacote de Dados (GB),2,20.00


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   CustomerID    1000 non-null   int64         
 1   fullName      1000 non-null   object        
 2   Genre         1000 non-null   object        
 3   PurchaseDate  1000 non-null   datetime64[ns]
 4   Product       1000 non-null   object        
 5   Quantity      1000 non-null   int64         
 6   Value         1000 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 54.8+ KB


In [5]:
df.isnull().sum()

,0
CustomerID,0
fullName,0
Genre,0
PurchaseDate,0
Product,0
Quantity,0
Value,0


In [6]:
df.duplicated().sum()

np.int64(0)

#Análise de clientes

In [16]:
df.columns
df_grouped = df.groupby(df.fullName).agg(

        Data_Ultima_Compra= ('PurchaseDate', 'max'),
        Compras_Realizadas= ('CustomerID', 'count'),
        Valor_Total= ('Value', 'sum'),
)
df_grouped = df_grouped.sort_values(by=['Compras_Realizadas', 'Valor_Total'], ascending=False)
df_grouped.describe().round(1)

,Data_Ultima_Compra,Compras_Realizadas,Valor_Total
count,199,199.0,199.0
mean,2024-04-01 16:24:07.236180992,5.0,316.2
min,2022-12-12 00:00:00,1.0,0.1
25%,2024-02-03 00:00:00,4.0,140.1
50%,2024-05-24 00:00:00,5.0,280.4
75%,2024-07-18 00:00:00,7.0,450.2
max,2024-08-24 00:00:00,11.0,940.0
std,NaN,2.2,220.3


In [32]:
df_grouped['Nivel'] = 'Bronze'
df_grouped['Nivel'] = df_grouped['Nivel'].case_when(
    [
        ((df_grouped.Valor_Total >= 450.2) & (df_grouped.Compras_Realizadas >= 7), 'Gold'),
        ((df_grouped.Valor_Total >= 140.1) & (df_grouped.Compras_Realizadas >= 4), 'Silver')
    ]
)

df_grouped

,Data_Ultima_Compra,Compras_Realizadas,Valor_Total,Nivel
fullName,,,,
Randall Wright,2024-08-21,11,790.35,Gold
Lisa Hernandez,2024-06-11,11,660.80,Gold
Shelly Burns,2024-04-25,11,620.35,Gold
Amanda Peterson,2024-06-11,10,940.05,Gold
Miranda Cortez,2024-06-20,10,840.30,Gold
...,...,...,...,...
Steven Rodriguez,2024-04-25,1,150.00,Bronze
Katherine Frank,2023-02-11,1,100.00,Bronze
Jessica James,2023-03-31,1,40.00,Bronze


In [62]:
dftest = df_grouped.Nivel.value_counts().reset_index()
dftest.columns = ['Nivel', 'Contagem']

dftest['Valor_Total'] = df_grouped.groupby('Nivel')['Valor_Total'].transform('sum').unique().round(2)

df_grouped.Valor_Total.sum()

# df_grouped.groupby('Nivel')['Valor_Total'].transform('sum').drop_duplicates()

np.float64(325252489.45000017)

#Função interessante: pd.qcut()

O  divide os dados com base nos valores dos quantis, em vez de limites fixos. Isso significa que cada intervalo conterá um número aproximadamente igual de observações. Aqui está a estrutura básica

In [63]:
import pandas as pd

# Dados exemplo
data = [1, 7, 5, 4, 6, 9, 3, 8, 2, 10]

# Dividir em 4 quantis (quartis)
categorias, limites = pd.qcut(data, q=4, labels=["Baixo", "Médio", "Alto", "Muito Alto"], retbins=True)

print("Categorias:")
print(categorias)

print("\nLimites dos intervalos:")
print(limites)

Categorias:
['Baixo', 'Alto', 'Médio', 'Médio', 'Alto', 'Muito Alto', 'Baixo', 'Muito Alto', 'Baixo', 'Muito Alto']
Categories (4, object): ['Baixo' < 'Médio' < 'Alto' < 'Muito Alto']

Limites dos intervalos:
[ 1.    3.25  5.5   7.75 10.  ]
